In [1]:
import os
print(os.getcwd())  # this is where Jupyter is looking from
#one folder back
os.chdir('..')
print(os.getcwd())

/root/n1_masters_thesis/model_training
/root/n1_masters_thesis


In [2]:
import pandas as pd
import numpy as np
from load_data import load_dataset, ImageDataset

from split import get_dataloaders
from custom_data_aug import get_transforms
from auto_aug import get_autoaugment_transforms

import torch
import torch.nn as nn
import torchvision
from config import CFG

from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter
from training import train  # or wherever your updated function lives

from acc_loss_plot import plot_training_curves

from torchvision.models import VGG16_Weights

In [3]:
# Load the dataframe
csv_path = "/root/n1_masters_thesis/filtered_df.csv"
image_dir = "/root/n1_masters_thesis/image_data/image_data"
df, idx_to_label = load_dataset(csv_path, image_dir)

In [4]:
print(df.head())

                                           file_name label  label_idx
0  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
1  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
2  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
3  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0
4  C:/Users/Christian/Desktop/N1_data/image_data/...    BC          0


In [4]:
#train_transforms, val_transforms = get_transforms()

#train_dataset, val_dataset, train_loader, val_loader = get_dataloaders(
#    df, train_transforms, val_transforms)

train_transforms, val_transforms = get_autoaugment_transforms()

train_dataset, val_dataset, train_loader, val_loader = get_dataloaders(
    df, train_transforms, val_transforms
)

In [6]:
def build_model(device: torch.device, num_classes: int = CFG.NUM_CLASSES) -> nn.Module:
    # Set seeds
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)
    
    model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    for layer in list(model.features.children())[-4:]:
        for param in layer.parameters():
            param.requires_grad = True

    model.avgpool = nn.AdaptiveAvgPool2d((1, 1))

    model.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 128),
        nn.ReLU(inplace=True),
        nn.Dropout(0.2),
        nn.Linear(128, num_classes)
    ).to(device)

    return model

In [7]:
cnn = build_model(device=CFG.DEVICE)

# View model summary
summary(
    model=cnn, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VGG (VGG)                                [16, 3, 224, 224]    [16, 34]             --                   Partial
├─Sequential (features)                  [16, 3, 224, 224]    [16, 512, 7, 7]      --                   Partial
│    └─Conv2d (0)                        [16, 3, 224, 224]    [16, 64, 224, 224]   (1,792)              False
│    └─ReLU (1)                          [16, 64, 224, 224]   [16, 64, 224, 224]   --                   --
│    └─Conv2d (2)                        [16, 64, 224, 224]   [16, 64, 224, 224]   (36,928)             False
│    └─ReLU (3)                          [16, 64, 224, 224]   [16, 64, 224, 224]   --                   --
│    └─MaxPool2d (4)                     [16, 64, 224, 224]   [16, 64, 112, 112]   --                   --
│    └─Conv2d (5)                        [16, 64, 112, 112]   [16, 128, 112, 112]  (73,856)             False
│    └─ReLU

In [8]:
# Define Loss Function
loss_fn = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# Define Optimizer
optimizer = torch.optim.Adam(
    cnn.parameters(),
    lr=CFG.LR
)

In [ ]:
writer = SummaryWriter(log_dir="runs/exp1_efficientnet")

session = train(
    model=cnn,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=CFG.EPOCHS,
    device=CFG.DEVICE,
    writer=writer
)

writer.close()


Epoch 1/30


  0%|          | 0/466 [00:00<?, ?it/s]

loss: 2.4849 - top1: 0.3174 - top5: 0.7276 - eval_loss: 1.8994 - eval_top1: 0.5180 - eval_top5: 0.9006

Epoch 2/30


  0%|          | 0/466 [00:00<?, ?it/s]

loss: 1.9283 - top1: 0.5081 - top5: 0.9026 - eval_loss: 1.6940 - eval_top1: 0.5817 - eval_top5: 0.9399

Epoch 3/30


...

  0%|          | 0/466 [00:00<?, ?it/s]

loss: 1.3938 - top1: 0.7411 - top5: 0.9706 - eval_loss: 1.4452 - eval_top1: 0.7071 - eval_top5: 0.9663

Epoch 9/30


  0%|          | 0/466 [00:00<?, ?it/s]

loss: 1.3443 - top1: 0.7641 - top5: 0.9728 - eval_loss: 1.4727 - eval_top1: 0.6831 - eval_top5: 0.9555

Epoch 10/30


  0%|          | 0/466 [00:00<?, ?it/s]

In [ ]:
# Convert history dict to DataFrame
session_history_df = pd.DataFrame(session)
session_history_df

In [ ]:
from acc_loss_plot import plot_training_curves
# Plot EfficientNet session training history 
plot_training_curves(session)